# Imports

In [1]:
import ipywidgets as widgets
import numpy as np
from IPython.display import display
import requests
import urllib.request
import json
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
from abc import ABC, abstractmethod
import aiohttp
import asyncio
from aiohttp import ClientSession
import re



In [16]:
class Parser(ABC):

    is_json = False

    @abstractmethod
    def get_price(self, response_text, card_name):
        pass

class HareruyaParser(Parser):
    name = 'Hareruya'
    url = 'https://www.hareruyamtg.com/en/products/search?suggest_type=all&product='

    def get_price(self, response_text, card_name):
        price_list = []
        soup = BeautifulSoup(response_text)
        for item in soup.find_all('div', attrs={'class': 'itemData'}):
            try:                
                card = card_name
                name =  re.search(fr'《({card})》', item.find("a", recursive=True).string, flags=re.IGNORECASE).groups(1)[0]
                stock = int(re.search(r"【NM Stock:(\d)】", item.find("p", attrs={'class': 'itemDetail__stock'}, recursive=True).string).groups(1)[0])
                price = int(re.search(r"¥ ([\d,]*)", item.find("p", attrs={'class': 'itemDetail__price'}, recursive=True).string).groups(1)[0].replace(',',''))
                if stock <= 0:
                    continue 
                price_list.append(round(price/77, 2))
            except AttributeError:
                continue
        return min(price_list) if len(price_list) > 0 else np.nan


class HobbymasterParser(Parser):
    name = 'Hobbymaster'
    url = 'https://hobbymaster.co.nz/cards/get-cards?foil=0&lang=&game=1&_search=true&sidx=set&sord=desc&name='
    is_json = True

    def get_price(self, response_text, card_name):
        data = response_text
        price_list = []
        if 'rows' in data:
            for item in data['rows']:
                if card_name.lower() in item['cell'][0].lower():
                    if item['cell'][12] == 0:
                        continue
                    else:
                        price = item['cell'][10]
                        price = float(price.replace('$', '').replace('!', ''))
                        price_list.append(price)
            if len(price_list) == 0:
                price = np.nan
            else:
                price = min(price_list)
        else:
            price = np.nan
        return price

class BaydragonParser(Parser):
    name = 'Baydragon'
    url = 'https://www.baydragon.co.nz/search/category/01?searchType=single&searchString='

    def get_price(self, response_text, card_name):
        soup = BeautifulSoup(response_text)
        div = soup.find('div', attrs={'class': 'tcgSingles'})
        table = div.find('table')
        tds = table.find_all('td')
        n = 0
        price_list = []
        for td in tds:
            n += 1
            if n > len(tds):
                break
            if 'NZ$' in td.text:
                if card_name in tds[n-6].text:
                    if int(tds[n].text) > 0:
                        price = float(td.text.replace('NZ$', ''))
                        price_list.append(price)
        if len(price_list) == 0:
            price = np.nan
        else:
            price = min(price_list)
        return price

class GoblinGamesParser(Parser):
    name = 'Goblin Games'
    url = 'https://goblingames.nz/search?q='

    def get_price(self, response_text, card_name):
        soup = BeautifulSoup(response_text)
        products = soup.find_all('div', attrs={'class': 'productCard__lower'})
        price_list = []
        for product in products:
            p_list = product.find_all('ul')
            for item in p_list:
                instock = item.find_all('li')
                if len(instock):
                    for i in instock:
                        title = i.attrs['data-producttitle']
                        price = int(i.attrs['data-price']) / 100
                    if card_name.lower() in title.lower():
                        price_list.append(price)
        if len(price_list) == 0:
            price = np.nan
        else:
            price = min(price_list)
        return price

class ShopifyParser(Parser):
    def __init__(self, url, name):
        self.url = url
        self.name = name
    def get_price(self, response_text, card_name):
    #     soup = BeautifulSoup(html_content, "lxml")
        soup = BeautifulSoup(response_text)
        products = soup.find_all('div', attrs={'class': 'product Norm'})
        price_list = []
        for product in products:
            title = product.find('p', attrs='productTitle').text.replace('\n', ' ').strip()
            price = product.find('p', attrs='productPrice').text.replace('\n', ' ').strip()
            if card_name.lower() in title.lower():
                if price == 'Varies':
                    prices = product.parent.find('div', attrs={'class': 'buyWrapper'}).find_all('p')
                    for p in prices:
                        try:
                            price = float(p.text.split('$')[1])
                        except:
                            continue
                        price_list.append(price)
                elif '$' in price:
                    try:
                        price = float(price.replace('$', ''))
                    except:
                        continue
                    price_list.append(price)
                else:
                    if price != 'Sold Out':
                        price_list.append(price)
        if len(price_list) == 0:
            price = np.nan
        else:
            price = min(price_list)

        return price



class CardSearcher():
    parsers = [
        # HobbymasterParser(),
        HareruyaParser(),
        BaydragonParser(),
        GoblinGamesParser(),
        ShopifyParser('https://spellboundgames.co.nz/search?q=', 'Spellbound'),
        ShopifyParser('https://magicatwillis.co.nz/search?q=', 'Magic at Willis'),
        ShopifyParser('https://ironknightgaming.co.nz/search?q=', 'Iron Knight Gaming'),
        ShopifyParser('https://mtgmagpie.com/search?q=', 'Magic Magpie')
    ]
    df_data = []
    cards = []

    async def run_search(self, cards):
        self.cards + cards
        await card_searcher.find_cards(cards)
        return self.display_prices()

    async def get_prices(self, card):
        prices = [card]
        async with aiohttp.ClientSession() as session:
            for parser in self.parsers:
                async with session.get(parser.url + card) as resp:
                    response = await resp.json() if parser.is_json else await resp.text()
                    prices.append(parser.get_price(response, card))
        return prices
                    
    async def find_cards(self, cards):
        for card in cards:
            price = await self.get_prices(card)
            self.df_data.append(price)

    def display_prices(self):
        # global results_df
        headers = ["Card"] + list(parser.name for parser in self.parsers)
        results_df = pd.DataFrame(self.df_data, columns =headers)
        return results_df
        

In [3]:
        
def highlight_min(data, color='green'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_min = data == data.min()
        return [attr if v else '' for v in is_min]
    else:  # from .apply(axis=None)
        is_min = data == data.min().min()
        return pd.DataFrame(np.where(is_min, attr, ''),
                            index=data.index, columns=data.columns)

async def start_card_search(button):
    # cards = card_input.value.split('\n')
    card_searcher = CardSearcher()
    cards = ["Sneak Attack"]
    out = widgets.Output()
    with out:
        df_output = await card_searcher.run_search(cards)
        display(df_output)

card_input = widgets.Textarea(placeholder='Enter a cards on new lines', rows=5)
go_button = widgets.Button(description='Find prices', button_style='')
results_output = widgets.Output()
table_output = widgets.Output()
display(card_input, go_button)
cards = card_input.value.split('\n')
go_button.on_click(start_card_search)

Textarea(value='', placeholder='Enter a cards on new lines', rows=5)

Button(description='Find prices', style=ButtonStyle())

In [22]:
card_searcher = CardSearcher()
card_searcher.cards = []
cards = ['Power Conduit', "Nesting Grounds", "Thieving Amalgam", "Bolas's Citadel", "Scroll of Fate"]
await card_searcher.run_search(cards)

,Card,Hareruya,Baydragon,Goblin Games,Spellbound,Magic at Willis,Iron Knight Gaming,Magic Magpie
0,"Akroma, Angel of Fury",3.90,1.60,2.00,3.0,1.4,2.30,3.05
1,"Akroma, Angel of Fury",3.90,1.60,2.00,3.0,1.4,2.30,3.05
2,Arcane Signet,7.79,4.65,41.50,7.2,NaN,NaN,NaN
3,Ashnod's Altar,10.39,NaN,NaN,NaN,NaN,NaN,NaN
4,Bankrupt in Blood,0.26,0.38,NaN,0.8,NaN,NaN,0.50
5,Blood Funnel,1.56,NaN,5.50,1.6,1.3,NaN,NaN
6,Bloodfell Caves,0.13,0.24,0.30,0.4,NaN,NaN,0.30
7,Bloodshot Cyclops,0.65,NaN,NaN,NaN,NaN,NaN,NaN
8,Bloodsoaked Altar,NaN,0.38,0.50,0.8,NaN,NaN,0.40
9,Breath of Fury,1.30,NaN,3.50,NaN,NaN,NaN,NaN


In [20]:
len(cards)


80

In [6]:

soup_text = BeautifulSoup(x.text)
products = soup.find_all('div', attrs={'class': 'productCard__lower'})
price_list = []
for product in products:
    p_list = product.find_all('ul')
    for item in p_list:
        instock = item.find_all('li')
        if len(instock):
            for i in instock:
                title = i.attrs['data-producttitle']
                price = int(i.attrs['data-price']) / 100
            if card_name.lower() in title.lower():
                price_list.append(price)
if len(price_list) == 0:
    price = np.nan
else:
    price = min(price_list)
return price


NameError: name 'x' is not defined

In [4]:
x = requests.get("https://www.hareruyamtg.com/en/products/search?suggest_type=all&product=Fervor")

Fervor 1 200
Fervor 6 400
Fervor 1 250
Fervor 1 12000
Fervor 5 300
Fervor 6 250
Fervor 3 400
Fervor 7 200


In [ ]:
df